In [1]:
import numpy as np
from scipy import stats, optimize
import pandas as pd

raw_csv = pd.read_csv("https://raw.githubusercontent.com/vanvisach/CCTVP/main/dataset/Raw.csv")
A1 = raw_csv.iloc[:, 1].values
A2 = raw_csv.iloc[:, 2].values

In [2]:
def weibull_mse_cdf(x, beta, eta):
    x = np.sort(np.asarray(x))
    n = len(x)

    i = np.arange(1, n + 1)
    F_emp = (i - 0.3) / (n + 0.4)  # Bernard median ranks
    F_fit = stats.weibull_min.cdf(x, c=beta, scale=eta, loc=0)

    return np.mean((F_fit - F_emp)**2)


In [3]:
import numpy as np
from scipy import stats
from scipy.special import gamma

# Weibull MLE (location fixed at 0)
beta_hat, _, eta_hat = stats.weibull_min.fit(A1, floc=0)

# MTTF estimate
mttf_hat = eta_hat * gamma(1 + 1 / beta_hat)

print("beta_hat:", beta_hat)
print("eta_hat :", eta_hat)
print("MTTF    :", mttf_hat)

beta_hat: 4.55779173853189
eta_hat : 86.64247768028977
MTTF    : 79.12621697897175


In [4]:
B = 2000
rng = np.random.default_rng(42)

beta_star = []
eta_star  = []
mttf_star = []
mse_star  = []

for _ in range(B):
    # Simulate from fitted Weibull
    x_star = stats.weibull_min.rvs(
        beta_hat, scale=eta_hat, size=len(A1), random_state=rng
    )

    # Refit Weibull
    b, _, e = stats.weibull_min.fit(x_star, floc=0)

    beta_star.append(b)
    eta_star.append(e)
    mttf_star.append(e * gamma(1 + 1 / b))

    mse_star.append(weibull_mse_cdf(x_star, b, e))


beta_star = np.array(beta_star)
eta_star  = np.array(eta_star)
mttf_star = np.array(mttf_star)
mse_star  = np.array(mse_star)


In [5]:
ci_beta = np.quantile(beta_star, [0.025, 0.975])
ci_eta  = np.quantile(eta_star,  [0.025, 0.975])
ci_mse  = np.quantile(mse_star,  [0.025, 0.975])

print("CI beta: ", ci_beta)
print("CI eta:  ", ci_eta)
print("CI MSE:  ", ci_mse)

CI beta:  [3.72412707 6.0569504 ]
CI eta:   [79.96182978 92.68506196]
CI MSE:   [0.00047008 0.00351158]


In [6]:
def fit_weibull_mom(x):
    x = np.asarray(x)
    m = x.mean()
    v = x.var(ddof=0)

    def equations(log_params):
        # optimize in log space to enforce positivity
        beta = np.exp(log_params[0])
        eta  = np.exp(log_params[1])

        mu = eta * gamma(1 + 1/beta)
        var = (eta**2) * (gamma(1 + 2/beta) - gamma(1 + 1/beta)**2)
        return np.array([mu - m, var - v])

    # initial guesses
    beta0 = 1.5
    eta0  = m  # rough
    sol = optimize.root(equations, x0=np.log([beta0, eta0]), method="hybr")
    if not sol.success:
        raise RuntimeError("MOM solve failed: " + sol.message)

    beta_hat = float(np.exp(sol.x[0]))
    eta_hat  = float(np.exp(sol.x[1]))
    return beta_hat, eta_hat

def fit_weibull_lsm(x):
    x = np.sort(np.asarray(x))
    n = len(x)
    i = np.arange(1, n+1)

    # Median ranks (Benard's approximation)
    F = (i - 0.3) / (n + 0.4)

    X = np.log(x)
    Y = np.log(-np.log(1 - F))

    A = np.vstack([X, np.ones_like(X)]).T
    slope, intercept = np.linalg.lstsq(A, Y, rcond=None)[0]

    beta_hat = float(slope)
    eta_hat  = float(np.exp(-intercept / slope))
    return beta_hat, eta_hat


def mse_cdf(x, beta, eta):
    x = np.sort(np.asarray(x))
    n = len(x)
    i = np.arange(1, n+1)

    F_emp = (i - 0.3) / (n + 0.4)   # empirical plotting position
    F_mod = stats.weibull_min.cdf(x, c=beta, scale=eta, loc=0)

    return float(np.mean((F_mod - F_emp)**2))

In [7]:
B = 2000
rng = np.random.default_rng(42)

out = {
    "MLE": {"beta": [], "eta": [], "mttf": [], "mse": []},
    "MOM": {"beta": [], "eta": [], "mttf": [], "mse": []},
    "LSM": {"beta": [], "eta": [], "mttf": [], "mse": []},
}

for _ in range(B):
    # For each Area
    x_star = stats.weibull_min.rvs(beta_hat, scale=eta_hat, size=len(A1), random_state=rng)

    # ---- MLE ----
    b_mle, _, e_mle = stats.weibull_min.fit(x_star, floc=0)
    out["MLE"]["beta"].append(b_mle)
    out["MLE"]["eta"].append(e_mle)
    out["MLE"]["mttf"].append(e_mle * gamma(1 + 1/b_mle))
    out["MLE"]["mse"].append(mse_cdf(x_star, b_mle, e_mle))

    # ---- MOM ----
    b_mom, e_mom = fit_weibull_mom(x_star)
    out["MOM"]["beta"].append(b_mom)
    out["MOM"]["eta"].append(e_mom)
    out["MOM"]["mttf"].append(e_mom * gamma(1 + 1/b_mom))
    out["MOM"]["mse"].append(mse_cdf(x_star, b_mom, e_mom))

    # ---- LSM ----
    b_lsm, e_lsm = fit_weibull_lsm(x_star)
    out["LSM"]["beta"].append(b_lsm)
    out["LSM"]["eta"].append(e_lsm)
    out["LSM"]["mttf"].append(e_lsm * gamma(1 + 1/b_lsm))
    out["LSM"]["mse"].append(mse_cdf(x_star, b_lsm, e_lsm))

for method in out:
    for k in out[method]:
        out[method][k] = np.asarray(out[method][k])


In [8]:
def ci95(a):
    return np.quantile(a, [0.025, 0.975])

summary = {}
for method in ["MOM", "MLE", "LSM"]:
    summary[method] = {
        "beta_hat": np.mean(out[method]["beta"]),
        "beta_ci":  ci95(out[method]["beta"]),
        "eta_hat":  np.mean(out[method]["eta"]),
        "eta_ci":   ci95(out[method]["eta"]),
        "mse_hat":  np.mean(out[method]["mse"]),
        "mse_ci":   ci95(out[method]["mse"]),
    }

summary


{'MOM': {'beta_hat': 4.729371116769651,
  'beta_ci': array([3.69521998, 6.08578288]),
  'eta_hat': 86.56958612403137,
  'eta_ci': array([79.94206387, 92.61881311]),
  'mse_hat': 0.0014215069658383415,
  'mse_ci': array([0.00046245, 0.00351719])},
 'MLE': {'beta_hat': 4.735004876211097,
  'beta_ci': array([3.72412707, 6.0569504 ]),
  'eta_hat': 86.58218109195887,
  'eta_ci': array([79.96182978, 92.68506196]),
  'mse_hat': 0.001419578721155533,
  'mse_ci': array([0.00047008, 0.00351158])},
 'LSM': {'beta_hat': 4.400125317274625,
  'beta_ci': array([3.08463641, 5.88551702]),
  'eta_hat': 87.10530378536514,
  'eta_ci': array([80.46434264, 93.56764509]),
  'mse_hat': 0.0017605169297992503,
  'mse_ci': array([0.0004626 , 0.00557545])}}

In [ ]:
import numpy as np

def weibull_F(t, beta, eta):
    return 1 - np.exp(-(t / eta) ** beta)

def weibull_R(t, beta, eta):
    return np.exp(-(t / eta) ** beta)

def weibull_h(t, beta, eta):
    return (beta / eta) * (t / eta) ** (beta - 1)


# bootstrap distributions
F_star = weibull_F(mttf_star, beta_star, eta_star)
R_star = weibull_R(mttf_star, beta_star, eta_star)
h_star = weibull_h(mttf_star, beta_star, eta_star)

# 95% CI (percentile)
F_ci = np.quantile(F_star, [0.025, 0.975])
R_ci = np.quantile(R_star, [0.025, 0.975])
h_ci = np.quantile(h_star, [0.025, 0.975])

# point estimates
F_hat = weibull_F(t0, beta_hat, eta_hat)
R_hat = weibull_R(t0, beta_hat, eta_hat)
h_hat = weibull_h(t0, beta_hat, eta_hat)

print("F(t):", F_hat, "95% CI:", F_ci)
print("R(t):", R_hat, "95% CI:", R_ci)
print("h(t):", h_hat, "95% CI:", h_ci)


A2

In [47]:
# Weibull MLE (location fixed at 0)
beta_hat, _, eta_hat = stats.weibull_min.fit(A2, floc=0)

# MTTF estimate
mttf_hat = eta_hat * gamma(1 + 1 / beta_hat)

print("beta_hat:", beta_hat)
print("eta_hat :", eta_hat)
print("MTTF    :", mttf_hat)

beta_hat: 4.388165956755772
eta_hat : 86.8206622725741
MTTF    : 79.11464369176781


In [48]:
B = 2000
rng = np.random.default_rng(42)

beta_star = []
eta_star  = []
mttf_star = []
mse_star  = []

for _ in range(B):
    # Simulate from fitted Weibull
    x_star = stats.weibull_min.rvs(
        beta_hat, scale=eta_hat, size=len(A2), random_state=rng
    )

    # Refit Weibull
    b, _, e = stats.weibull_min.fit(x_star, floc=0)

    beta_star.append(b)
    eta_star.append(e)
    mttf_star.append(e * gamma(1 + 1 / b))

    mse_star.append(weibull_mse_cdf(x_star, b, e))


beta_star = np.array(beta_star)
eta_star  = np.array(eta_star)
mttf_star = np.array(mttf_star)
mse_star  = np.array(mse_star)


In [49]:
B = 2000
rng = np.random.default_rng(42)

out = {
    "MLE": {"beta": [], "eta": [], "mttf": [], "mse": []},
    "MOM": {"beta": [], "eta": [], "mttf": [], "mse": []},
    "LSM": {"beta": [], "eta": [], "mttf": [], "mse": []},
}

for _ in range(B):
    # For each Area
    x_star = stats.weibull_min.rvs(beta_hat, scale=eta_hat, size=len(A2), random_state=rng)

    # ---- MLE ----
    b_mle, _, e_mle = stats.weibull_min.fit(x_star, floc=0)
    out["MLE"]["beta"].append(b_mle)
    out["MLE"]["eta"].append(e_mle)
    out["MLE"]["mttf"].append(e_mle * gamma(1 + 1/b_mle))
    out["MLE"]["mse"].append(mse_cdf(x_star, b_mle, e_mle))

    # ---- MOM ----
    b_mom, e_mom = fit_weibull_mom(x_star)
    out["MOM"]["beta"].append(b_mom)
    out["MOM"]["eta"].append(e_mom)
    out["MOM"]["mttf"].append(e_mom * gamma(1 + 1/b_mom))
    out["MOM"]["mse"].append(mse_cdf(x_star, b_mom, e_mom))

    # ---- LSM ----
    b_lsm, e_lsm = fit_weibull_lsm(x_star)
    out["LSM"]["beta"].append(b_lsm)
    out["LSM"]["eta"].append(e_lsm)
    out["LSM"]["mttf"].append(e_lsm * gamma(1 + 1/b_lsm))
    out["LSM"]["mse"].append(mse_cdf(x_star, b_lsm, e_lsm))

for method in out:
    for k in out[method]:
        out[method][k] = np.asarray(out[method][k])


In [50]:
def ci95(a):
    return np.quantile(a, [0.025, 0.975])

summary = {}
for method in ["MOM", "MLE", "LSM"]:
    summary[method] = {
        "beta_hat": np.mean(out[method]["beta"]),
        "beta_ci":  ci95(out[method]["beta"]),
        "eta_hat":  np.mean(out[method]["eta"]),
        "eta_ci":   ci95(out[method]["eta"]),
        "mse_hat":  np.mean(out[method]["mse"]),
        "mse_ci":   ci95(out[method]["mse"]),
        "mttf_hat": np.mean(out[method]["mttf"]),
        "mttf_ci":  ci95(out[method]["mttf"])
    }

summary


{'MOM': {'beta_hat': np.float64(4.552549651039637),
  'beta_ci': array([3.55911031, 5.84374262]),
  'eta_hat': np.float64(86.74780009860041),
  'eta_ci': array([79.85334328, 93.05407486]),
  'mse_hat': np.float64(0.0014154409880144747),
  'mse_ci': array([0.00046212, 0.00349609]),
  'mttf_hat': np.float64(79.1869456769515),
  'mttf_ci': array([72.44205188, 85.44930842])},
 'MLE': {'beta_hat': np.float64(4.558783733468432),
  'beta_ci': array([3.58552477, 5.83152791]),
  'eta_hat': np.float64(86.76036917091241),
  'eta_ci': array([79.87806587, 93.1179792 ]),
  'mse_hat': np.float64(0.001419578408455236),
  'mse_ci': array([0.00047008, 0.00351154]),
  'mttf_hat': np.float64(79.20839546782135),
  'mttf_ci': array([72.45440002, 85.46054143])},
 'LSM': {'beta_hat': np.float64(4.2363673529636),
  'beta_ci': array([2.96983655, 5.66647774]),
  'eta_hat': np.float64(87.30496021271112),
  'eta_ci': array([80.3995847 , 94.03917647]),
  'mse_hat': np.float64(0.0017605169297992495),
  'mse_ci': arr

In [51]:
import numpy as np

def weibull_F(t, beta, eta):
    return 1 - np.exp(-(t / eta) ** beta)

def weibull_R(t, beta, eta):
    return np.exp(-(t / eta) ** beta)

def weibull_h(t, beta, eta):
    return (beta / eta) * (t / eta) ** (beta - 1)


# bootstrap distributions
F_star = weibull_F(mttf_star, beta_star, eta_star)
R_star = weibull_R(mttf_star, beta_star, eta_star)
h_star = weibull_h(mttf_star, beta_star, eta_star)

# 95% CI (percentile)
F_ci = np.quantile(F_star, [0.025, 0.975])
R_ci = np.quantile(R_star, [0.025, 0.975])
h_ci = np.quantile(h_star, [0.025, 0.975])

# point estimates
F_hat = weibull_F(t0, beta_hat, eta_hat)
R_hat = weibull_R(t0, beta_hat, eta_hat)
h_hat = weibull_h(t0, beta_hat, eta_hat)

print("F(t):", F_hat, "95% CI:", F_ci)
print("R(t):", R_hat, "95% CI:", R_ci)
print("h(t):", h_hat, "95% CI:", h_ci)


F(t): 1.2282655337036985e-05 95% CI: [0.47255031 0.49735605]
R(t): 0.999987717344663 95% CI: [0.50264395 0.52744969]
h(t): 8.166463790706247e-06 95% CI: [0.03197565 0.04621927]
